In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import Counter
import pandas as pd
import numpy as np
import re

In [58]:
# open labelled data

file = open("Fake.csv")
df_fakes = pd.read_csv(file)

file2 = open("True.csv")
df_true = pd.read_csv(file2)

In [59]:
# split data, 75 percent for training

#train
train_fakes = df_fakes["title"][:17610]
train_true = df_true["title"][:16062]

#test
test_fakes = df_fakes["title"][17610:]
test_true = df_true["title"][16062:]

In [60]:
# create dictionary of words for true article titles and fake ones
dd_true = defaultdict(Counter)
dd_fake = defaultdict(Counter)

# fake articles
for i, fake_title in enumerate(train_fakes):
    # tokenize words
    lTokens = []
    sToken = ""
    for c in fake_title:
        if re.match("[a-zA-Z0-9]", str(c)) != None or c == "\'" or c == "_" or c == '-':
            sToken += c
        else:
            if sToken != "":
                lTokens.append(sToken)
                sToken = ""
            if c.strip() != "":
                lTokens.append(str(c.strip()))
    if sToken != "":
        lTokens.append(sToken)

    # update word to dictionary
    for word in lTokens:
        dd_fake[i][word] = dd_fake[i][word] + 1
        
# true articles
for ii, true_title in enumerate(train_true):
    # tokenize words
    lTokens = []
    sToken = ""
    for c in true_title:
        if re.match("[a-zA-Z0-9]", str(c)) != None or c == "\'" or c == "_" or c == '-':
            sToken += c
        else:
            if sToken != "":
                lTokens.append(sToken)
                sToken = ""
            if c.strip() != "":
                lTokens.append(str(c.strip()))
    if sToken != "":
        lTokens.append(sToken)

    # update word to dictionary
    for word in lTokens:
        dd_true[ii][word] = dd_true[ii][word] + 1

In [61]:
all_titles = []
for tf, tt in zip(test_fakes, test_true):
    l = [tf, 0]
    l2 = [tt, 1]
    all_titles.append(l)
    all_titles.append(l2)

In [62]:
#### CREATE NAIVE BAYES FROM HERE 

#Priors
total_docs = len(dd_fake) + len(dd_true)
prior_fake = len(dd_fake)/total_docs
prior_true = len(dd_true)/total_docs

#Posteriors Structures Fake
total_freq_per_word_fake = Counter()
for doc, words in dd_fake.items():
    for word, count in words.items():
        total_freq_per_word_fake[word] += count       
total_freq_fake = sum(total_freq_per_word_fake.values())   

#Posteriors Structures True
total_freq_per_word_true = Counter()
for doc, words in dd_true.items():
    for word, count in words.items():
        total_freq_per_word_true[word] += count       
total_freq_true = sum(total_freq_per_word_true.values())   

In [63]:
# TEST CLASSIFIER

TP = 0
TN = 0
FP = 0
FN = 0

# TEST with fake titles below first

for title in all_titles:
    # tokenize words
    lTokens = []
    sToken = ""
    for c in title[0]:
        if re.match("[a-zA-Z0-9]", str(c)) != None or c == "\'" or c == "_" or c == '-':
            sToken += c
        else:
            if sToken != "":
                lTokens.append(sToken)
                sToken = ""
            if c.strip() != "":
                lTokens.append(str(c.strip()))
    if sToken != "":
        lTokens.append(sToken)

#Classifier
    total_prob_fake = 1
    total_prob_true = 1
    for word in lTokens:

        # check Positive Structures
        freq_word_fake = total_freq_per_word_fake[word]
        word_prob_fake = 0
        if freq_word_fake == 0:
            word_prob_fake = 0
        else:
            word_prob_fake = freq_word_fake/total_freq_fake
        if word_prob_fake == 0:
            word_prob_fake = (np.finfo(float).eps)
        total_prob_fake = total_prob_fake * word_prob_fake

        #check Negative Structures
        freq_word_true = total_freq_per_word_true[word]
        word_prob_true = 0
        if freq_word_true == 0:
            word_prob_true = 0
        else:
            word_prob_true = freq_word_true/total_freq_true
        if word_prob_true == 0:
            word_prob_true = (np.finfo(float).eps)
        total_prob_true = total_prob_true * word_prob_true

    prob_doc_fake = total_prob_fake * prior_fake
    prob_doc_true = total_prob_true * prior_true

    #pred label
    probs = [prob_doc_fake, prob_doc_true]
    m = max(probs)
    index = probs.index(m)
    if index == 0:
        pred_label = "False"
    elif index == 1:
        pred_label = "True"
    
    # actual label
    if title[1] == 0:
        actual_label ="False"
    if title[1] == 1:
        actual_label ="True"

    
    if pred_label == "False" and actual_label == "False":
        TP += 1
    if pred_label == "True" and actual_label == "True":
        TN += 1
    if pred_label == "False" and actual_label == "True":
        FP += 1
    if pred_label == "True" and actual_label == "False":
        FN += 1

# Accuracy Calculations
print()
print("Basic Classifier Statistics:")
print()

Accuracy = (TP+TN)/(TP+FP+FN+TN)
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1_Score = 2*(Recall * Precision) / (Recall + Precision)

print(Accuracy)
print(Precision)
print(Recall)
print(F1_Score)


Basic Classifier Statistics:

0.9952380952380953
0.9983089064261556
0.9921568627450981
0.995223377353189


In [64]:
# MORE TESTING, enter list of items:  item == [title, label] 1 for true, 0 for fake

def classifier(titles):
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    # TEST with fake titles below first

    for title in titles:
        # tokenize words
        lTokens = []
        sToken = ""
        for c in title[0]:
            if re.match("[a-zA-Z0-9]", str(c)) != None or c == "\'" or c == "_" or c == '-':
                sToken += c
            else:
                if sToken != "":
                    lTokens.append(sToken)
                    sToken = ""
                if c.strip() != "":
                    lTokens.append(str(c.strip()))
        if sToken != "":
            lTokens.append(sToken)

    #Classifier
        total_prob_fake = 1
        total_prob_true = 1
        for word in lTokens:

            # check Positive Structures
            freq_word_fake = total_freq_per_word_fake[word]
            word_prob_fake = 0
            if freq_word_fake == 0:
                word_prob_fake = 0
            else:
                word_prob_fake = freq_word_fake/total_freq_fake
            if word_prob_fake == 0:
                word_prob_fake = (np.finfo(float).eps)
            total_prob_fake = total_prob_fake * word_prob_fake

            #check Negative Structures
            freq_word_true = total_freq_per_word_true[word]
            word_prob_true = 0
            if freq_word_true == 0:
                word_prob_true = 0
            else:
                word_prob_true = freq_word_true/total_freq_true
            if word_prob_true == 0:
                word_prob_true = (np.finfo(float).eps)
            total_prob_true = total_prob_true * word_prob_true

        prob_doc_fake = total_prob_fake * prior_fake
        prob_doc_true = total_prob_true * prior_true

        #pred label
        probs = [prob_doc_fake, prob_doc_true]
        m = max(probs)
        index = probs.index(m)
        if index == 0:
            pred_label = "False"
        elif index == 1:
            pred_label = "True"

        
        
        # actual label
        if title[1] == 0:
            actual_label ="False"
        if title[1] == 1:
            actual_label ="True"


        if pred_label == "False" and actual_label == "False":
            TP += 1
        if pred_label == "True" and actual_label == "True":
            TN += 1
        if pred_label == "False" and actual_label == "True":
            FP += 1
        if pred_label == "True" and actual_label == "False":
            FN += 1
        
        print(title, pred_label)
    # Accuracy Calculations
    print()
    print("Basic Classifier Statistics:")
    print()

    Accuracy = (TP+TN)/(TP+FP+FN+TN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_Score = 2*(Recall * Precision) / (Recall + Precision)

    print(Accuracy)
    print(Precision)
    print(Recall)
    print(F1_Score)

In [65]:
# perfect predictions
inputt = [["China's Financial Sector", 1], ["CRAZY PEOPLE like to Lie",0], 
         ["Congress approves legislation", 1], 

classifier(inputt)

["China's Financial Sector", 1] True
['CRAZY PEOPLE like to Lie', 0] False
['Congress approves legislation', 1] True

Basic Classifier Statistics:

0.875
1.0
0.8
0.888888888888889
